In [ ]:
# ref: https://coolpuppy.readthedocs.io/en/latest/Examples/Walkthrough_API.html

In [ ]:
import os
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline")
print(os.getcwd())

In [ ]:
! find . -name *rep12.merged.1k.zoomify.mcool

In [ ]:
! find . -name *WT*rep1.*1k.zoomify.mcool

In [ ]:
! find . -name *WT*rep2.*1k.zoomify.mcool

# 读取需要用到的文件

## 读取 mcool 文件

In [ ]:
import cooler
resolution = 10_000
# mcool 文件
nora_wt_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_mcool_file = "./Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
ren_wt_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
ren_iaa_mcool_file = "./Ren_IAA/Ren_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
wn_bpk25_mcool_file = "./WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.1k.zoomify.mcool"
# 生成 r10k cool 互作矩阵对象
nora_wt_r10k_clr = cooler.Cooler(f'{nora_wt_mcool_file}::/resolutions/{resolution}')
nora_iaa_r10k_clr = cooler.Cooler(f'{nora_iaa_mcool_file}::/resolutions/{resolution}')
ren_wt_r10k_clr = cooler.Cooler(f'{ren_wt_mcool_file}::/resolutions/{resolution}')
ren_iaa_r10k_clr = cooler.Cooler(f'{ren_iaa_mcool_file}::/resolutions/{resolution}')
wn_bpk25_r10k_clr = cooler.Cooler(f'{wn_bpk25_mcool_file}::/resolutions/{resolution}')


In [ ]:
# Nora/Ren WT rep1/2 mcool 后续用于 divide pileup 表示 WT 两次重复之间没啥差异，用 IAA 或 BPK25 处理后有明显差异
nora_wt_rep1_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
nora_wt_rep2_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC_rep2.nodups.pairs.1k.zoomify.mcool"
ren_wt_rep1_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
ren_wt_rep2_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC_rep2.nodups.pairs.1k.zoomify.mcool"
# 生成 r10k cool 互作矩阵对象
nora_wt_rep1_r10k_clr = cooler.Cooler(f'{nora_wt_rep1_mcool_file}::/resolutions/{resolution}')
nora_wt_rep2_r10k_clr = cooler.Cooler(f'{nora_wt_rep2_mcool_file}::/resolutions/{resolution}')
ren_wt_rep1_r10k_clr = cooler.Cooler(f'{ren_wt_rep1_mcool_file}::/resolutions/{resolution}')
ren_wt_rep2_r10k_clr = cooler.Cooler(f'{ren_wt_rep2_mcool_file}::/resolutions/{resolution}')


## 读取 chrom view 文件

In [ ]:
import pandas as pd
mm10_chr_view_df = pd.read_csv("./mm10.chrom.view.tsv", sep="\t", header=0)
print(mm10_chr_view_df.shape)
mm10_chr_view_df.head()

# 计算生成 expected_cis_df

In [ ]:
from cooltools import expected_cis
# Calculate expected interactions for chromosome arms
nora_wt_r10k_expected_cis_df = expected_cis(clr=nora_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)
nora_iaa_r10k_expected_cis_df = expected_cis(clr=nora_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)
ren_wt_r10k_expected_cis_df = expected_cis(clr=ren_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)
ren_iaa_r10k_expected_cis_df = expected_cis(clr=ren_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)
wn_bpk25_r10k_expected_cis_df = expected_cis(clr=wn_bpk25_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)


In [ ]:
# Nora/Ren WT rep1/2 expected_cis_df
nora_wt_rep1_r10k_expected_cis_df = expected_cis(clr=nora_wt_rep1_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)
nora_wt_rep2_r10k_expected_cis_df = expected_cis(clr=nora_wt_rep2_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)
ren_wt_rep1_r10k_expected_cis_df = expected_cis(clr=ren_wt_rep1_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)
ren_wt_rep2_r10k_expected_cis_df = expected_cis(clr=ren_wt_rep2_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=128)


# 读取 CTCF summit及 insulation boundaries 信息并对其进行分类

## 读取 CTCF summit 文件

In [ ]:
import pandas as pd
mESC_WT_CTCF_summit_df = pd.read_csv("./mESC_WT_d0246_union_CTCF.summit.sorted.bed", sep="\t", header=None)
mESC_WT_CTCF_summit_df.columns = ["chrom", "start", "end", "fold_enrichment"]
print(mESC_WT_CTCF_summit_df.shape)
mESC_WT_CTCF_summit_df.head()

In [ ]:
# 为 summit 添加四分类标签
mESC_WT_CTCF_summit_df['quartile_FC_score'] = pd.qcut(mESC_WT_CTCF_summit_df['fold_enrichment'], 4, labels=False) + 1
print(mESC_WT_CTCF_summit_df.shape)
mESC_WT_CTCF_summit_df.head()

In [ ]:
mESC_WT_CTCF_summit_df.quartile_FC_score.value_counts()

## 读取 Ren_WT r10k boundaries 结果文件

In [ ]:
! find . -name *.rep12.merged.mcool_r10k.insulation_table.tsv

In [ ]:
ren_wt_r10k_insTab_file = "./Ren_WT/Ren_mESC_WT_HiC.rep12.merged.mcool_r10k.insulation_table.tsv"
ren_wt_r10k_insTab_df = pd.read_csv(ren_wt_r10k_insTab_file, sep="\t", header=0)
print(ren_wt_r10k_insTab_df.shape)
ren_wt_r10k_insTab_df.head()

## insulation boundary 筛选及有用信息列提取

In [ ]:
usecols = ["chrom","start","end","log2_insulation_score_150000","n_valid_pixels_150000","boundary_strength_150000"]
ren_wt_r10k_ibdr_df = ren_wt_r10k_insTab_df[ren_wt_r10k_insTab_df.is_boundary_150000==True][usecols]
ren_wt_r10k_ibdr_df.columns = ["chrom","start","end","log2_insulation_score","n_valid_pixels","boundary_strength"]
ren_wt_r10k_ibdr_df = ren_wt_r10k_ibdr_df.reset_index().drop(columns="index")
print(ren_wt_r10k_ibdr_df.shape)
ren_wt_r10k_ibdr_df.head()

In [ ]:
ren_wt_r10k_ibdr_df.describe().style.format("{:.2f}")

## 添加 boundary_strength 四分类列

In [ ]:
ren_wt_r10k_ibdr_df["strength_quartile"] = pd.qcut(ren_wt_r10k_ibdr_df['boundary_strength'], 4, labels=False) + 1
print(ren_wt_r10k_ibdr_df.shape)
ren_wt_r10k_ibdr_df.head()

In [ ]:
ren_wt_r10k_ibdr_df.strength_quartile.value_counts()

##  添加 CTCF BS 分类列

In [ ]:
# 自定义函数判断 boundary 是否有 CTCF 结合
def add_CBS_type_for_bdr_bed(bed_row, ctcf_bs_bed, ext_len=25):
    chrom, start, end = bed_row["chrom"], bed_row["start"], bed_row["end"]
    CTCF_BS = "No"
    CTCF_FC = 0
    for bed_row in ctcf_bs_bed.itertuples(index=True, name='Pandas'):
        cbs_chr = bed_row.chrom
        cbs_start = bed_row.start - ext_len
        cbs_end = bed_row.end + ext_len
        fc_enrichment = bed_row.fold_enrichment
        if chrom == cbs_chr and (start<=cbs_start<end or start<cbs_end<=end):
            CTCF_BS = "Yes"
            CTCF_FC = fc_enrichment
            break
    return CTCF_BS, CTCF_FC

ren_wt_r10k_ibdr_df[["CTCF_BS", "CTCF_FC"]] = ren_wt_r10k_ibdr_df.apply(add_CBS_type_for_bdr_bed, axis=1, 
                                                                            args=(mESC_WT_CTCF_summit_df, 25), 
                                                                            result_type='expand')
ren_wt_r10k_ibdr_df.head()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(5, 5))
# ax1
ax1 = fig.add_subplot(1, 1, 1)
sns.boxplot(ren_wt_r10k_ibdr_df, x="CTCF_BS", y="boundary_strength", hue="CTCF_BS", ax=ax1)
plt.title("Ren_WT_r10k_insulation_boundaries_strength_boxplot", loc="center")
plt.show()
fig.savefig('Ren_WT_r10k_insulation_boundaries_strength_boxplot.groupby_CTCF_BS.pdf')
fig.savefig('Ren_WT_r10k_insulation_boundaries_strength_boxplot.groupby_CTCF_BS.png')


In [ ]:
CBS_boundary_df = ren_wt_r10k_ibdr_df[ren_wt_r10k_ibdr_df.CTCF_BS=="Yes"][["boundary_strength", "CTCF_FC"]]
print(CBS_boundary_df.shape)
CBS_boundary_df.head()

In [ ]:
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt

boundary_strength = CBS_boundary_df.boundary_strength
CTCF_log2FC = np.log2(CBS_boundary_df.CTCF_FC)
pearsonr , prs_pval = stats.pearsonr(boundary_strength, CTCF_log2FC)

xy = np.vstack([boundary_strength, CTCF_log2FC])
kde = stats.gaussian_kde(xy)(xy)

fig = plt.figure(figsize=(6, 6))
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# ax1.plot([0, 3], [0, 3], ls='--', lw=1.2, c="limegreen", alpha=0.7)
ax1.scatter(x=CTCF_log2FC, y=boundary_strength, s=2, c=kde, cmap='coolwarm', edgecolors='none')
ax1.text(0.3, 2.6, r'$  r = $'+str(format(pearsonr, '.6f')+"\np = "+str(format(prs_pval, '.2e'))))
ax1.set_xlim(0, 5)
ax1.set_ylim(0, 3)
ax1.set_xlabel("CTCF_log2FC")
ax1.set_ylabel("boundary_strength")

plt.show()
fig.savefig('Ren_WT_r10k_insulation_boundaries_with_CBS.bdr_strength_and_CTCF_log2FC_corrlation_scatter_plot.colour_use_kde.pdf')
fig.savefig('Ren_WT_r10k_insulation_boundaries_with_CBS.bdr_strength_and_CTCF_log2FC_corrlation_scatter_plot.colour_use_kde.png')


# Ren_WT r10k boundaries 不分组 pileup

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_ibdr_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
nora_wt_r10k_clr_ibdr_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT.png")

## Nora_WT_rep1

In [ ]:
from coolpuppy import coolpup
nora_wt_rep1_r10k_clr_ibdr_pup = coolpup.pileup(clr=nora_wt_rep1_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
nora_wt_rep1_r10k_clr_ibdr_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_rep1_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep1.png")


## Nora_WT_rep2

In [ ]:
from coolpuppy import coolpup
nora_wt_rep2_r10k_clr_ibdr_pup = coolpup.pileup(clr=nora_wt_rep2_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
nora_wt_rep2_r10k_clr_ibdr_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_rep2_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep2.png")


## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_ibdr_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
nora_iaa_r10k_clr_ibdr_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_iaa_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_IAA.png")

## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_ibdr_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
ren_wt_r10k_clr_ibdr_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT.png")


## Ren_WT_rep1

In [ ]:
from coolpuppy import coolpup
ren_wt_rep1_r10k_clr_ibdr_pup = coolpup.pileup(clr=ren_wt_rep1_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
ren_wt_rep1_r10k_clr_ibdr_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_rep1_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep1.png")

## Ren_WT_rep2

In [ ]:
from coolpuppy import coolpup
ren_wt_rep2_r10k_clr_ibdr_pup = coolpup.pileup(clr=ren_wt_rep2_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
ren_wt_rep2_r10k_clr_ibdr_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_rep2_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep2.png")

## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_ibdr_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
ren_iaa_r10k_clr_ibdr_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_iaa_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_IAA.png")


## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_ibdr_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=128)
wn_bpk25_r10k_clr_ibdr_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_ibdr_pup,
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25.png")


## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
# nora_wt_r10k_clr_ibdr_pup
# nora_iaa_r10k_clr_ibdr_pup
# wn_bpk25_r10k_clr_ibdr_pup
nora_wn_ibdr_pup_merged_df = pd.concat([nora_wt_r10k_clr_ibdr_pup, nora_iaa_r10k_clr_ibdr_pup, wn_bpk25_r10k_clr_ibdr_pup], ignore_index=True)
nora_wn_ibdr_pup_merged_df


In [ ]:
nora_wn_ibdr_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25"]
nora_wn_ibdr_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ibdr_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WN.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WN.png")

## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
# ren_wt_r10k_clr_ibdr_pup
# ren_iaa_r10k_clr_ibdr_pup
# wn_bpk25_r10k_clr_ibdr_pup
ren_wn_ibdr_pup_merged_df = pd.concat([ren_wt_r10k_clr_ibdr_pup, ren_iaa_r10k_clr_ibdr_pup, wn_bpk25_r10k_clr_ibdr_pup], ignore_index=True)
ren_wn_ibdr_pup_merged_df


In [ ]:
ren_wn_ibdr_pup_merged_df["group"] = ["Ren_WT", "Ren_IAA", "WN_BPK25"]
ren_wn_ibdr_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wn_ibdr_pup_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WN.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WN.png")


## all experiment merge in one

In [ ]:
nora_wn_ren_ibdr_pup_merged_df = pd.concat([nora_wt_r10k_clr_ibdr_pup, nora_iaa_r10k_clr_ibdr_pup, wn_bpk25_r10k_clr_ibdr_pup, ren_iaa_r10k_clr_ibdr_pup, ren_wt_r10k_clr_ibdr_pup], ignore_index=True)
nora_wn_ren_ibdr_pup_merged_df


In [ ]:
nora_wn_ren_ibdr_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"]
nora_wn_ren_ibdr_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ren_ibdr_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WN_Ren.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WN_Ren.png")


## Nora_WT_rep2_vs_rep1

In [ ]:
# nora_wt_rep1_r10k_clr_ibdr_pup
# nora_wt_rep2_r10k_clr_ibdr_pup
from coolpuppy.lib.puputils import divide_pups
ibdr_pup_nora_wt_rep2_vs_rep1 = divide_pups(nora_wt_rep2_r10k_clr_ibdr_pup, nora_wt_rep1_r10k_clr_ibdr_pup)
ibdr_pup_nora_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_nora_wt_rep2_vs_rep1,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})\n Nora_WT_rep2_vs_rep1')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep2_vs_rep1.png")


## Ren_WT_rep2_vs_rep1

In [ ]:
# ren_wt_rep1_r10k_clr_ibdr_pup
# ren_wt_rep2_r10k_clr_ibdr_pup
from coolpuppy.lib.puputils import divide_pups
ibdr_pup_ren_wt_rep2_vs_rep1 = divide_pups(ren_wt_rep2_r10k_clr_ibdr_pup, ren_wt_rep1_r10k_clr_ibdr_pup)
ibdr_pup_ren_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_ren_wt_rep2_vs_rep1,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})\n Ren_WT_rep2_vs_rep1')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep2_vs_rep1.png")


## Nora_IAA_vs_WT

In [ ]:
# nora_iaa_r10k_clr_ibdr_pup
# nora_wt_r10k_clr_ibdr_pup
from coolpuppy.lib.puputils import divide_pups
ibdr_pup_nora_iaa_vs_wt = divide_pups(nora_iaa_r10k_clr_ibdr_pup, nora_wt_r10k_clr_ibdr_pup)
ibdr_pup_nora_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_nora_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})\n Nora_IAA_vs_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_IAA_vs_WT.png")

## Ren_IAA_vs_WT

In [ ]:
# ren_iaa_r10k_clr_ibdr_pup
# ren_wt_r10k_clr_ibdr_pup
from coolpuppy.lib.puputils import divide_pups
ibdr_pup_ren_iaa_vs_wt = divide_pups(ren_iaa_r10k_clr_ibdr_pup, ren_wt_r10k_clr_ibdr_pup)
ibdr_pup_ren_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_ren_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})\n Ren_IAA_vs_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_IAA_vs_WT.png")


## WN_BPK25_vs_Nora_WT

In [ ]:
# wn_bpk25_r10k_clr_ibdr_pup
# nora_wt_r10k_clr_ibdr_pup
from coolpuppy.lib.puputils import divide_pups
ibdr_pup_wn_bpk25_vs_nora_wt = divide_pups(wn_bpk25_r10k_clr_ibdr_pup, nora_wt_r10k_clr_ibdr_pup)
ibdr_pup_wn_bpk25_vs_nora_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_wn_bpk25_vs_nora_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]}) \n WN_BPK25_vs_Nora_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25_vs_Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25_vs_Nora_WT.png")


## WN_BPK25_vs_Ren_WT

In [ ]:
# wn_bpk25_r10k_clr_ibdr_pup
# ren_wt_r10k_clr_ibdr_pup
from coolpuppy.lib.puputils import divide_pups
ibdr_pup_wn_bpk25_vs_ren_wt = divide_pups(wn_bpk25_r10k_clr_ibdr_pup, ren_wt_r10k_clr_ibdr_pup)
ibdr_pup_wn_bpk25_vs_ren_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_wn_bpk25_vs_ren_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]}) \n WN_BPK25_vs_Ren_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25_vs_Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25_vs_Ren_WT.png")


## merge 4 divide pileup in one

In [ ]:
ibdr_pup_divide_merged_df = pd.concat([ibdr_pup_nora_iaa_vs_wt, ibdr_pup_ren_iaa_vs_wt, ibdr_pup_wn_bpk25_vs_nora_wt, ibdr_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
ibdr_pup_divide_merged_df


In [ ]:
ibdr_pup_divide_merged_df.insert(0, "comparation", ["IAA_vs_WT", "IAA_vs_WT", "BPK25_vs_WT", "BPK25_vs_WT"] )
ibdr_pup_divide_merged_df.insert(1, "WT_source", ["Nora", "Ren", "Nora", "Ren"] )
ibdr_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_divide_merged_df,
                rows="comparation",
                row_order=["IAA_vs_WT", "BPK25_vs_WT"],
                rownames=["IAA_vs_WT", "BPK25_vs_WT"],
                cols='WT_source',
                col_order=["Nora", "Ren"],
                colnames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_4comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_4comparation.exp_col.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_divide_merged_df,
                cols="comparation",
                col_order=["IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora", "Ren"],
                rownames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_4comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_4comparation.exp_row.png")


## merge 6 divide pileup in one

In [ ]:
ibdr_pup_divide_merged_df = pd.concat([ibdr_pup_nora_wt_rep2_vs_rep1, ibdr_pup_ren_wt_rep2_vs_rep1, ibdr_pup_nora_iaa_vs_wt, ibdr_pup_ren_iaa_vs_wt, ibdr_pup_wn_bpk25_vs_nora_wt, ibdr_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
ibdr_pup_divide_merged_df


In [ ]:
ibdr_pup_divide_merged_df.insert(0, "comp_group", ["WT_rep2vs1", "WT_rep2vs1", "IAA_vs_WT", "IAA_vs_WT", "BPK25_vs_WT", "BPK25_vs_WT"] )
ibdr_pup_divide_merged_df.insert(1, "WT_source", ["Nora", "Ren", "Nora", "Ren", "Nora", "Ren"] )
ibdr_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_divide_merged_df,
                rows="comp_group",
                row_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rownames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                cols='WT_source',
                col_order=["Nora", "Ren"],
                colnames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_6comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_6comparation.exp_col.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_divide_merged_df,
                cols="comp_group",
                col_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora", "Ren"],
                rownames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_6comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_6comparation.exp_row.png")


## merge plot for Nora WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_divide_merged_df,
                cols="comp_group",
                col_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora"],
                rownames=["use_Nora_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_Nora_WN_3comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_Nora_WN_3comparation.exp_row.png")


## merge plot for Ren WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ibdr_pup_divide_merged_df,
                cols="comp_group",
                col_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Ren"],
                rownames=["use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.suptitle(f'Ren_WT_r10k_boundaries ({ren_wt_r10k_ibdr_df.shape[0]})')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_Ren_WN_3comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_Ren_WN_3comparation.exp_row.png")


# Ren_WT r10k boundaries 根据 CBS 分两组 pileup

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_wt_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT.png")


## Nora_WT_rep1

In [ ]:
from coolpuppy import coolpup
nora_wt_rep1_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=nora_wt_rep1_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep1_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_wt_rep1_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep1.png")

## Nora_WT_rep2

In [ ]:
from coolpuppy import coolpup
nora_wt_rep2_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=nora_wt_rep2_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep2_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_wt_rep2_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep2.png")

## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_iaa_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=nora_iaa_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_IAA.png")

## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_wt_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT.png")

## Ren_WT_rep1

In [ ]:
from coolpuppy import coolpup
ren_wt_rep1_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=ren_wt_rep1_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep1_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_wt_rep1_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep1.png")


## Ren_WT_rep2

In [ ]:
from coolpuppy import coolpup
ren_wt_rep2_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=ren_wt_rep2_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep2_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_wt_rep2_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep2.png")


## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_iaa_r10k_clr_bdr_CBS2c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=ren_iaa_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_IAA.png")


## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_bdr_CBS2c_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["CTCF_BS1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
wn_bpk25_r10k_clr_bdr_CBS2c_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_bdr_CBS2c_pup,
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25.png")


## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
boundary_CBS2c_pup_nora_wn_merged_df = pd.concat([nora_wt_r10k_clr_bdr_CBS2c_pup, nora_iaa_r10k_clr_bdr_CBS2c_pup, wn_bpk25_r10k_clr_bdr_CBS2c_pup], ignore_index=True)
boundary_CBS2c_pup_nora_wn_merged_df


In [ ]:
boundary_CBS2c_pup_nora_wn_merged_df["group"] = ["Nora_WT"]*3 + ["Nora_IAA"]*3 + ["WN_BPK25"]*3
boundary_CBS2c_pup_nora_wn_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundary_CBS2c_pup_nora_wn_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundary_CBS2c_pup_nora_wn_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rows="CTCF_BS1",
                row_order=["all", "Yes", "No"],
                rownames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN.exp_col.png")


## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
boundary_CBS2c_pup_ren_wn_merged_df = pd.concat([ren_wt_r10k_clr_bdr_CBS2c_pup, ren_iaa_r10k_clr_bdr_CBS2c_pup, wn_bpk25_r10k_clr_bdr_CBS2c_pup], ignore_index=True)
boundary_CBS2c_pup_ren_wn_merged_df


In [ ]:
boundary_CBS2c_pup_ren_wn_merged_df["group"] = ["Ren_WT"]*3 + ["Ren_IAA"]*3 + ["WN_BPK25"]*3
boundary_CBS2c_pup_ren_wn_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundary_CBS2c_pup_ren_wn_merged_df,
                rows="group",
                row_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rownames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WN.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WN.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundary_CBS2c_pup_ren_wn_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rows="CTCF_BS1",
                row_order=["all", "Yes", "No"],
                rownames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WN.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WN.exp_col.png")


## all experiment merge in one

In [ ]:
boundary_CBS2c_pup_nora_wn_ren_merged_df = pd.concat([nora_wt_r10k_clr_bdr_CBS2c_pup, nora_iaa_r10k_clr_bdr_CBS2c_pup, wn_bpk25_r10k_clr_bdr_CBS2c_pup, ren_iaa_r10k_clr_bdr_CBS2c_pup, ren_wt_r10k_clr_bdr_CBS2c_pup], ignore_index=True)
boundary_CBS2c_pup_nora_wn_ren_merged_df


In [ ]:
boundary_CBS2c_pup_nora_wn_ren_merged_df["group"] = ["Nora_WT"]*3 + ["Nora_IAA"]*3 + ["WN_BPK25"]*3 + ["Ren_IAA"]*3 + ["Ren_WT"]*3
boundary_CBS2c_pup_nora_wn_ren_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundary_CBS2c_pup_nora_wn_ren_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                cols="CTCF_BS1",
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN_Ren.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN_Ren.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundary_CBS2c_pup_nora_wn_ren_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rows="CTCF_BS1",
                row_order=["all", "Yes", "No"],
                rownames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN_Ren.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN_Ren.exp_col.png")


## Nora_WT_rep2_vs_rep1

In [ ]:
# nora_wt_rep1_r10k_clr_bdr_CBS2c_pup
# nora_wt_rep2_r10k_clr_bdr_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    rep1_pup = nora_wt_rep1_r10k_clr_bdr_CBS2c_pup[nora_wt_rep1_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    rep2_pup = nora_wt_rep2_r10k_clr_bdr_CBS2c_pup[nora_wt_rep2_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    rep2_vs_rep1 = divide_pups(rep2_pup, rep1_pup)
    all_CTCF_BS_group_divide_pup.append(rep2_vs_rep1)
boundaries_CBS2c_pup_nora_wt_rep2_vs_rep1 = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
boundaries_CBS2c_pup_nora_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_nora_wt_rep2_vs_rep1,
            cols='CTCF_BS1',
            col_order=["all", "Yes", "No"],
            colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Nora_WT_rep2_vs_rep1')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep2_vs_rep1.png")


## Ren_WT_rep2_vs_rep1

In [ ]:
# ren_wt_rep1_r10k_clr_bdr_CBS2c_pup
# ren_wt_rep2_r10k_clr_bdr_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    rep1_pup = ren_wt_rep1_r10k_clr_bdr_CBS2c_pup[ren_wt_rep1_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    rep2_pup = ren_wt_rep2_r10k_clr_bdr_CBS2c_pup[ren_wt_rep2_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    rep2_vs_rep1 = divide_pups(rep2_pup, rep1_pup)
    all_CTCF_BS_group_divide_pup.append(rep2_vs_rep1)
boundaries_CBS2c_pup_ren_wt_rep2_vs_rep1 = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
boundaries_CBS2c_pup_ren_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_ren_wt_rep2_vs_rep1,
            cols='CTCF_BS1',
            col_order=["all", "Yes", "No"],
            colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_rep2_vs_rep1')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep2_vs_rep1.png")


## Nora_IAA_vs_WT

In [ ]:
# nora_wt_r10k_clr_bdr_CBS2c_pup
# nora_iaa_r10k_clr_bdr_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    iaa_pup = nora_iaa_r10k_clr_bdr_CBS2c_pup[nora_iaa_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    wt_pup = nora_wt_r10k_clr_bdr_CBS2c_pup[nora_wt_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    iaa_vs_wt = divide_pups(iaa_pup, wt_pup)
    all_CTCF_BS_group_divide_pup.append(iaa_vs_wt)
boundaries_CBS2c_pup_nora_iaa_vs_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
boundaries_CBS2c_pup_nora_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_nora_iaa_vs_wt,
            cols='CTCF_BS1',
            col_order=["all", "Yes", "No"],
            colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Nora_IAA_vs_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_IAA_vs_WT.png")


## Ren_IAA_vs_WT

In [ ]:
# ren_wt_r10k_clr_bdr_CBS2c_pup
# ren_iaa_r10k_clr_bdr_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    iaa_pup = ren_iaa_r10k_clr_bdr_CBS2c_pup[ren_iaa_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    wt_pup = ren_wt_r10k_clr_bdr_CBS2c_pup[ren_wt_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    iaa_vs_wt = divide_pups(iaa_pup, wt_pup)
    all_CTCF_BS_group_divide_pup.append(iaa_vs_wt)
boundaries_CBS2c_pup_ren_iaa_vs_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
boundaries_CBS2c_pup_ren_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_ren_iaa_vs_wt,
            cols='CTCF_BS1',
            col_order=["all", "Yes", "No"],
            colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_IAA_vs_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_IAA_vs_WT.png")


## WN_BPK25_vs_Nora_WT

In [ ]:
# nora_wt_r10k_clr_bdr_CBS2c_pup
# wn_bpk25_r10k_clr_bdr_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    wn_bpk25_pup = wn_bpk25_r10k_clr_bdr_CBS2c_pup[wn_bpk25_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    nora_wt_pup = nora_wt_r10k_clr_bdr_CBS2c_pup[nora_wt_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    wn_bpk25_vs_nora_wt = divide_pups(wn_bpk25_pup, nora_wt_pup)
    all_CTCF_BS_group_divide_pup.append(wn_bpk25_vs_nora_wt)
boundaries_CBS2c_pup_wn_bpk25_vs_nora_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
boundaries_CBS2c_pup_wn_bpk25_vs_nora_wt


In [ ]:

import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_wn_bpk25_vs_nora_wt,
            cols='CTCF_BS1',
            col_order=["all", "Yes", "No"],
            colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'WN_BPK25_vs_Nora_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25_vs_Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25_vs_Nora_WT.png")


## WN_BPK25_vs_Ren_WT

In [ ]:
# ren_wt_r10k_clr_bdr_CBS2c_pup
# wn_bpk25_r10k_clr_bdr_CBS2c_pup
from coolpuppy.lib.puputils import divide_pups
all_CTCF_BS_group_divide_pup = []
for CTCF_BS_group in ["all", "Yes", "No"]:
    wn_bpk25_pup = wn_bpk25_r10k_clr_bdr_CBS2c_pup[wn_bpk25_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    ren_wt_pup = ren_wt_r10k_clr_bdr_CBS2c_pup[ren_wt_r10k_clr_bdr_CBS2c_pup.CTCF_BS1==CTCF_BS_group]
    wn_bpk25_vs_ren_wt = divide_pups(wn_bpk25_pup, ren_wt_pup)
    all_CTCF_BS_group_divide_pup.append(wn_bpk25_vs_ren_wt)
boundaries_CBS2c_pup_wn_bpk25_vs_ren_wt = pd.concat(all_CTCF_BS_group_divide_pup, ignore_index=True)
boundaries_CBS2c_pup_wn_bpk25_vs_ren_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_wn_bpk25_vs_ren_wt,
            cols='CTCF_BS1',
            col_order=["all", "Yes", "No"],
            colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'WN_BPK25_vs_Ren_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25_vs_Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25_vs_Ren_WT.png")


## merge 4 divide pileup in one

In [ ]:
boundaries_CBS2c_pup_divide_merged_df = pd.concat([boundaries_CBS2c_pup_nora_iaa_vs_wt, boundaries_CBS2c_pup_ren_iaa_vs_wt, boundaries_CBS2c_pup_wn_bpk25_vs_nora_wt, boundaries_CBS2c_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
boundaries_CBS2c_pup_divide_merged_df


In [ ]:
boundaries_CBS2c_pup_divide_merged_df.insert(0, "divide_group", ["Nora_IAA_vs_WT"]*3+["Ren_IAA_vs_WT"]*3+["WN_BPK25_vs_Nora_WT"]*3+["WN_BPK25_vs_Ren_WT"]*3)
boundaries_CBS2c_pup_divide_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_divide_merged_df,
                rows="divide_group",
                row_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rownames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                cols='CTCF_BS1',
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_4comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_4comparation.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rows='CTCF_BS1',
                row_order=["all", "Yes", "No"],
                rownames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_4comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_4comparation.exp_col.png")


## merge 6 divide pileup in one

In [ ]:
boundaries_CBS2c_pup_divide_merged_df = pd.concat([boundaries_CBS2c_pup_nora_wt_rep2_vs_rep1, boundaries_CBS2c_pup_ren_wt_rep2_vs_rep1, boundaries_CBS2c_pup_nora_iaa_vs_wt, boundaries_CBS2c_pup_ren_iaa_vs_wt, boundaries_CBS2c_pup_wn_bpk25_vs_nora_wt, boundaries_CBS2c_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
boundaries_CBS2c_pup_divide_merged_df


In [ ]:
boundaries_CBS2c_pup_divide_merged_df.insert(0, "divide_group", ["Nora_WT_rep2vs1"]*3+["Ren_WT_rep2vs1"]*3+["Nora_IAA_vs_WT"]*3+["Ren_IAA_vs_WT"]*3+["WN_BPK25_vs_Nora_WT"]*3+["WN_BPK25_vs_Ren_WT"]*3)
boundaries_CBS2c_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_divide_merged_df,
                rows="divide_group",
                row_order=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rownames=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                cols='CTCF_BS1',
                col_order=["all", "Yes", "No"],
                colnames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_6comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_6comparation.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rows='CTCF_BS1',
                row_order=["all", "Yes", "No"],
                rownames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_6comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_6comparation.exp_col.png")


## merge plot for Nora WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT","WN_BPK25_vs_Nora_WT"],
                colnames=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"],
                rows='CTCF_BS1',
                row_order=["all", "Yes", "No"],
                rownames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_Nora_WN_3comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_Nora_WN_3comparation.exp_col.png")


## merge plot for Ren WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
all_boundaries_count = ren_wt_r10k_ibdr_df.shape[0]
CBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["Yes"]
nonCBS_boundaries_count = ren_wt_r10k_ibdr_df.CTCF_BS.value_counts()["No"]
plotpup.plot(pupsdf=boundaries_CBS2c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT","WN_BPK25_vs_Ren_WT"],
                colnames=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"],
                rows='CTCF_BS1',
                row_order=["all", "Yes", "No"],
                rownames=[f'Ren_WT_r10k_boundaries\n({all_boundaries_count})', f'boundaries_with_CBS\n({CBS_boundaries_count})', f'boundaries_without_CBS\n({nonCBS_boundaries_count})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_Ren_WN_3comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_Ren_WN_3comparation.exp_col.png")


# Ren_WT r10k boundaries 根据 boundary strength 分四组 pileup

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=nora_wt_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT.png")


## Nora_WT_rep1

In [ ]:
from coolpuppy import coolpup
nora_wt_rep1_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=nora_wt_rep1_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep1_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=nora_wt_rep1_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep1.png")


## Nora_WT_rep2

In [ ]:
from coolpuppy import coolpup
nora_wt_rep2_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=nora_wt_rep2_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_wt_rep2_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=nora_wt_rep2_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep2.png")


## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
nora_iaa_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=nora_iaa_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_IAA.png")


## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=ren_wt_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT.png")


## Ren_WT_rep1

In [ ]:
from coolpuppy import coolpup
ren_wt_rep1_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=ren_wt_rep1_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep1_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=ren_wt_rep1_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep1.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep1.png")


## Ren_WT_rep2

In [ ]:
from coolpuppy import coolpup
ren_wt_rep2_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=ren_wt_rep2_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_wt_rep2_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=ren_wt_rep2_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep2.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep2.png")


## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
ren_iaa_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=ren_iaa_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_IAA.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_IAA.png")


## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_bdr_sQ4c_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=ren_wt_r10k_ibdr_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            local=True,
                                            groupby=["strength_quartile1"],
                                            flank=300_000,
                                            min_diag=0,
                                            nproc=32)
wn_bpk25_r10k_clr_bdr_sQ4c_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_bdr_sQ4c_pup,
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25.png")


## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
boundary_sQ4c_pup_nora_wn_merged_df = pd.concat([nora_wt_r10k_clr_bdr_sQ4c_pup, nora_iaa_r10k_clr_bdr_sQ4c_pup, wn_bpk25_r10k_clr_bdr_sQ4c_pup], ignore_index=True)
boundary_sQ4c_pup_nora_wn_merged_df


In [ ]:
boundary_sQ4c_pup_nora_wn_merged_df["group"] = ["Nora_WT"]*5 + ["Nora_IAA"]*5 + ["WN_BPK25"]*5
boundary_sQ4c_pup_nora_wn_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundary_sQ4c_pup_nora_wn_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundary_sQ4c_pup_nora_wn_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                rows="strength_quartile1",
                row_order=["all", 1, 2, 3, 4],
                rownames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN.exp_col.png")


## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
boundary_sQ4c_pup_ren_wn_merged_df = pd.concat([ren_wt_r10k_clr_bdr_sQ4c_pup, ren_iaa_r10k_clr_bdr_sQ4c_pup, wn_bpk25_r10k_clr_bdr_sQ4c_pup], ignore_index=True)
boundary_sQ4c_pup_ren_wn_merged_df


In [ ]:
boundary_sQ4c_pup_ren_wn_merged_df["group"] = ["Ren_WT"]*5 + ["Ren_IAA"]*5 + ["WN_BPK25"]*5
boundary_sQ4c_pup_ren_wn_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundary_sQ4c_pup_ren_wn_merged_df,
                rows="group",
                row_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rownames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WN.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WN.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundary_sQ4c_pup_ren_wn_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                rows="strength_quartile1",
                row_order=["all", 1, 2, 3, 4],
                rownames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WN.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WN.exp_col.png")


## all experiment merge in one

In [ ]:
boundary_sQ4c_pup_nora_wn_ren_merged_df = pd.concat([nora_wt_r10k_clr_bdr_sQ4c_pup, nora_iaa_r10k_clr_bdr_sQ4c_pup, wn_bpk25_r10k_clr_bdr_sQ4c_pup, ren_iaa_r10k_clr_bdr_sQ4c_pup, ren_wt_r10k_clr_bdr_sQ4c_pup], ignore_index=True)
boundary_sQ4c_pup_nora_wn_ren_merged_df


In [ ]:
boundary_sQ4c_pup_nora_wn_ren_merged_df["group"] = ["Nora_WT"]*5 + ["Nora_IAA"]*5 + ["WN_BPK25"]*5 + ["Ren_IAA"]*5 + ["Ren_WT"]*5
boundary_sQ4c_pup_nora_wn_ren_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundary_sQ4c_pup_nora_wn_ren_merged_df,
                rows="group",
                row_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rownames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                cols="strength_quartile1",
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN_Ren.exp_row.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN_Ren.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundary_sQ4c_pup_nora_wn_ren_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                rows="strength_quartile1",
                row_order=["all", 1, 2, 3, 4],
                rownames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN_Ren.exp_col.pdf")
plt.savefig("coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN_Ren.exp_col.png")


## Nora_WT_rep2_vs_rep1

In [ ]:
# nora_wt_rep1_r10k_clr_bdr_sQ4c_pup
# nora_wt_rep2_r10k_clr_bdr_sQ4c_pup
from coolpuppy.lib.puputils import divide_pups
all_sQ4c_group_divide_pup = []
for sQ4c_group in ["all", 1, 2, 3, 4]:
    rep1_pup = nora_wt_rep1_r10k_clr_bdr_sQ4c_pup[nora_wt_rep1_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    rep2_pup = nora_wt_rep2_r10k_clr_bdr_sQ4c_pup[nora_wt_rep2_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    rep2_vs_rep1 = divide_pups(rep2_pup, rep1_pup)
    all_sQ4c_group_divide_pup.append(rep2_vs_rep1)
boundaries_sQ4c_pup_nora_wt_rep2_vs_rep1 = pd.concat(all_sQ4c_group_divide_pup, ignore_index=True)
boundaries_sQ4c_pup_nora_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_nora_wt_rep2_vs_rep1,
            cols='strength_quartile1',
            col_order=["all", 1, 2, 3, 4],
            colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Nora_WT_rep2_vs_rep1')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep2_vs_rep1.png")


## Ren_WT_rep2_vs_rep1

In [ ]:
# ren_wt_rep1_r10k_clr_bdr_sQ4c_pup
# ren_wt_rep2_r10k_clr_bdr_sQ4c_pup
from coolpuppy.lib.puputils import divide_pups
all_sQ4c_group_divide_pup = []
for sQ4c_group in ["all", 1, 2, 3, 4]:
    rep1_pup = ren_wt_rep1_r10k_clr_bdr_sQ4c_pup[ren_wt_rep1_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    rep2_pup = ren_wt_rep2_r10k_clr_bdr_sQ4c_pup[ren_wt_rep2_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    rep2_vs_rep1 = divide_pups(rep2_pup, rep1_pup)
    all_sQ4c_group_divide_pup.append(rep2_vs_rep1)
boundaries_sQ4c_pup_ren_wt_rep2_vs_rep1 = pd.concat(all_sQ4c_group_divide_pup, ignore_index=True)
boundaries_sQ4c_pup_ren_wt_rep2_vs_rep1


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_ren_wt_rep2_vs_rep1,
            cols='strength_quartile1',
            col_order=["all", 1, 2, 3, 4],
            colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_WT_rep2_vs_rep1')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep2_vs_rep1.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep2_vs_rep1.png")


## Nora_IAA_vs_WT

In [ ]:
# nora_wt_r10k_clr_bdr_sQ4c_pup
# nora_iaa_r10k_clr_bdr_sQ4c_pup
from coolpuppy.lib.puputils import divide_pups
all_sQ4c_group_divide_pup = []
for sQ4c_group in ["all", 1, 2, 3, 4]:
    nora_iaa_pup = nora_iaa_r10k_clr_bdr_sQ4c_pup[nora_iaa_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    nora_wt_pup = nora_wt_r10k_clr_bdr_sQ4c_pup[nora_wt_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    nora_iaa_vs_wt = divide_pups(nora_iaa_pup, nora_wt_pup)
    all_sQ4c_group_divide_pup.append(nora_iaa_vs_wt)
boundaries_sQ4c_pup_nora_iaa_vs_wt = pd.concat(all_sQ4c_group_divide_pup, ignore_index=True)
boundaries_sQ4c_pup_nora_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_nora_iaa_vs_wt,
            cols='strength_quartile1',
            col_order=["all", 1, 2, 3, 4],
            colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Nora_IAA_vs_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_IAA_vs_WT.png")


## Ren_IAA_vs_WT

In [ ]:
# ren_wt_r10k_clr_bdr_sQ4c_pup
# ren_iaa_r10k_clr_bdr_sQ4c_pup
from coolpuppy.lib.puputils import divide_pups
all_sQ4c_group_divide_pup = []
for sQ4c_group in ["all", 1, 2, 3, 4]:
    ren_iaa_pup = ren_iaa_r10k_clr_bdr_sQ4c_pup[ren_iaa_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    ren_wt_pup = ren_wt_r10k_clr_bdr_sQ4c_pup[ren_wt_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    ren_iaa_vs_wt = divide_pups(ren_iaa_pup, ren_wt_pup)
    all_sQ4c_group_divide_pup.append(ren_iaa_vs_wt)
boundaries_sQ4c_pup_ren_iaa_vs_wt = pd.concat(all_sQ4c_group_divide_pup, ignore_index=True)
boundaries_sQ4c_pup_ren_iaa_vs_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_ren_iaa_vs_wt,
            cols='strength_quartile1',
            col_order=["all", 1, 2, 3, 4],
            colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'Ren_IAA_vs_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_IAA_vs_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_IAA_vs_WT.png")


## WN_BPK25_vs_Nora_WT

In [ ]:
# nora_wt_r10k_clr_bdr_sQ4c_pup
# wn_bpk25_r10k_clr_bdr_sQ4c_pup
from coolpuppy.lib.puputils import divide_pups
all_sQ4c_group_divide_pup = []
for sQ4c_group in ["all", 1, 2, 3, 4]:
    wn_bpk25_pup = wn_bpk25_r10k_clr_bdr_sQ4c_pup[wn_bpk25_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    nora_wt_pup = nora_wt_r10k_clr_bdr_sQ4c_pup[nora_wt_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    wn_bpk25_vs_nora_wt = divide_pups(wn_bpk25_pup, nora_wt_pup)
    all_sQ4c_group_divide_pup.append(wn_bpk25_vs_nora_wt)
boundaries_sQ4c_pup_wn_bpk25_vs_nora_wt = pd.concat(all_sQ4c_group_divide_pup, ignore_index=True)
boundaries_sQ4c_pup_wn_bpk25_vs_nora_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_wn_bpk25_vs_nora_wt,
            cols='strength_quartile1',
            col_order=["all", 1, 2, 3, 4],
            colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'WN_BPK25_vs_Nora_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25_vs_Nora_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25_vs_Nora_WT.png")


## WN_BPK25_vs_Ren_WT

In [ ]:
# ren_wt_r10k_clr_bdr_sQ4c_pup
# wn_bpk25_r10k_clr_bdr_sQ4c_pup
from coolpuppy.lib.puputils import divide_pups
all_sQ4c_group_divide_pup = []
for sQ4c_group in ["all", 1, 2, 3, 4]:
    wn_bpk25_pup = wn_bpk25_r10k_clr_bdr_sQ4c_pup[wn_bpk25_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    ren_wt_pup = ren_wt_r10k_clr_bdr_sQ4c_pup[ren_wt_r10k_clr_bdr_sQ4c_pup.strength_quartile1==sQ4c_group]
    wn_bpk25_vs_ren_wt = divide_pups(wn_bpk25_pup, ren_wt_pup)
    all_sQ4c_group_divide_pup.append(wn_bpk25_vs_ren_wt)
boundaries_sQ4c_pup_wn_bpk25_vs_ren_wt = pd.concat(all_sQ4c_group_divide_pup, ignore_index=True)
boundaries_sQ4c_pup_wn_bpk25_vs_ren_wt


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_wn_bpk25_vs_ren_wt,
            cols='strength_quartile1',
            col_order=["all", 1, 2, 3, 4],
            colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.suptitle(f'WN_BPK25_vs_Ren_WT')
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25_vs_Ren_WT.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25_vs_Ren_WT.png")


## merge 4 divide pileup in one

In [ ]:
boundaries_sQ4c_pup_divide_merged_df = pd.concat([boundaries_sQ4c_pup_nora_iaa_vs_wt, boundaries_sQ4c_pup_ren_iaa_vs_wt, boundaries_sQ4c_pup_wn_bpk25_vs_nora_wt, boundaries_sQ4c_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
boundaries_sQ4c_pup_divide_merged_df


In [ ]:
boundaries_sQ4c_pup_divide_merged_df.insert(0, "divide_group", ["Nora_IAA_vs_WT"]*5+["Ren_IAA_vs_WT"]*5+["WN_BPK25_vs_Nora_WT"]*5+["WN_BPK25_vs_Ren_WT"]*5)
boundaries_sQ4c_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_divide_merged_df,
                rows="divide_group",
                row_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rownames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                cols='strength_quartile1',
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_4comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_4comparation.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rows='strength_quartile1',
                row_order=["all", 1, 2, 3, 4],
                rownames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_4comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_4comparation.exp_col.png")


## merge 6 divide pileup in one

In [ ]:
boundaries_sQ4c_pup_divide_merged_df = pd.concat([boundaries_sQ4c_pup_nora_wt_rep2_vs_rep1, boundaries_sQ4c_pup_ren_wt_rep2_vs_rep1, boundaries_sQ4c_pup_nora_iaa_vs_wt, boundaries_sQ4c_pup_ren_iaa_vs_wt, boundaries_sQ4c_pup_wn_bpk25_vs_nora_wt, boundaries_sQ4c_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
boundaries_sQ4c_pup_divide_merged_df


In [ ]:
boundaries_sQ4c_pup_divide_merged_df.insert(0, "divide_group", ["Nora_WT_rep2vs1"]*5+["Ren_WT_rep2vs1"]*5+["Nora_IAA_vs_WT"]*5+["Ren_IAA_vs_WT"]*5+["WN_BPK25_vs_Nora_WT"]*5+["WN_BPK25_vs_Ren_WT"]*5)
boundaries_sQ4c_pup_divide_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_divide_merged_df,
                rows="divide_group",
                row_order=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rownames=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                cols='strength_quartile1',
                col_order=["all", 1, 2, 3, 4],
                colnames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_6comparation.exp_row.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_6comparation.exp_row.png")


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Nora_WT_rep2vs1", "Ren_WT_rep2vs1", "Nora_IAA_vs_WT", "Ren_IAA_vs_WT", "WN_BPK25_vs_Nora_WT", "WN_BPK25_vs_Ren_WT"],
                rows='strength_quartile1',
                row_order=["all", 1, 2, 3, 4],
                rownames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_6comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_6comparation.exp_col.png")


## merge plot for Nora WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"],
                colnames=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"],
                rows='strength_quartile1',
                row_order=["all", 1, 2, 3, 4],
                rownames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_Nora_WN_3comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_Nora_WN_3comparation.exp_col.png")


## merge plot for Ren WN 3 comparation

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
Qcounts = ren_wt_r10k_ibdr_df.strength_quartile.value_counts()
plotpup.plot(pupsdf=boundaries_sQ4c_pup_divide_merged_df,
                cols="divide_group",
                col_order=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"],
                rows='strength_quartile1',
                row_order=["all", 1, 2, 3, 4],
                rownames=[f'Ren_WT_r10k_boundaries\n({ren_wt_r10k_ibdr_df.shape[0]})', f'strength_Q1_boundaries\n({Qcounts[1]})', f'strength_Q2_boundaries\n({Qcounts[2]})', f'strength_Q3_boundaries\n({Qcounts[3]})', f'strength_Q4_boundaries\n({Qcounts[4]})'],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_Ren_WN_3comparation.exp_col.pdf")
plt.savefig("coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_Ren_WN_3comparation.exp_col.png")


# 制作结果汇总 PPT

In [ ]:
! ls -1 coolpuppy_*for_Ren_WT_r10k_boundaries*.pdf

In [ ]:
! ls -1 coolpuppy_*for_Ren_WT_r10k_boundaries*.png

## add boundaries_noClassified

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN

Ren_WT_r10k_boundaries_noC_Nora_WT_rep1_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep1.png"
Ren_WT_r10k_boundaries_noC_Nora_WT_rep2_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep2.png"
Ren_WT_r10k_boundaries_noC_Ren_WT_rep1_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep1.png"
Ren_WT_r10k_boundaries_noC_Ren_WT_rep2_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep2.png"

Ren_WT_r10k_boundaries_noC_Nora_WT_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT.png"
Ren_WT_r10k_boundaries_noC_Nora_IAA_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_IAA.png"
Ren_WT_r10k_boundaries_noC_Ren_WT_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT.png"
Ren_WT_r10k_boundaries_noC_Ren_IAA_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_IAA.png"
Ren_WT_r10k_boundaries_noC_WN_BPK25_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25.png"

Ren_WT_r10k_boundaries_noC_Nora_WN_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WN.png"
Ren_WT_r10k_boundaries_noC_Ren_WN_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WN.png"
Ren_WT_r10k_boundaries_noC_Nora_WN_Ren_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WN_Ren.png"

def add_text_anno(slide, anno_text, left, top, width, height):
    # 插入文本框
    text_box = slide.shapes.add_textbox(left, top, width, height)
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_box.text_frame.text = anno_text
    text_box.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    # 设置字体大小和加粗
    text_font = text_box.text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(6)
    text_font.bold = True


# 创建 PPT 并插入图片
my_ppt = Presentation()
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

# slide1
slide1 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_file, left, top, width, height)
slide1.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Nora_WT_rep1_png, Cm(3.51), Cm(0.61), Cm(6.53), Cm(6.53))
slide1.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Nora_WT_rep2_png, Cm(15.37), Cm(0.61), Cm(6.53), Cm(6.53))
slide1.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Ren_WT_rep1_png, Cm(3.51), Cm(7.76), Cm(6.53), Cm(6.53))
slide1.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Ren_WT_rep2_png, Cm(15.37), Cm(7.76), Cm(6.53), Cm(6.53))
# 插入文本注释
add_text_anno(slide1, "Ren_WT_r10k_boundaries_noC_Nora_WT_rep1_pileup_plot", Cm(3.21), Cm(0.05), Cm(7.14), Cm(0.5))
add_text_anno(slide1, "Ren_WT_r10k_boundaries_noC_Nora_WT_rep2_pileup_plot", Cm(15.07), Cm(0.05), Cm(7.14), Cm(0.5))
add_text_anno(slide1, "Ren_WT_r10k_boundaries_noC_Ren_WT_rep1_pileup_plot", Cm(3.21), Cm(7.19), Cm(7.14), Cm(0.5))
add_text_anno(slide1, "Ren_WT_r10k_boundaries_noC_Ren_WT_rep2_pileup_plot", Cm(15.07), Cm(7.19), Cm(7.14), Cm(0.5))

# slide2
slide2 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide2.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Nora_WT_png, Cm(0.83), Cm(0.61), Cm(6.53), Cm(6.53))
slide2.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Nora_IAA_png, Cm(9.38), Cm(0.61), Cm(6.53), Cm(6.53))
slide2.shapes.add_picture(Ren_WT_r10k_boundaries_noC_WN_BPK25_png, Cm(18.03), Cm(0.61), Cm(6.53), Cm(6.53))
slide2.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Ren_WT_png, Cm(0.83), Cm(7.76), Cm(6.53), Cm(6.53))
slide2.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Ren_IAA_png, Cm(9.38), Cm(7.76), Cm(6.53), Cm(6.53))
# 插入文本注释
add_text_anno(slide2, "Ren_WT_r10k_boundaries_noC_Nora_WT_pileup_plot", Cm(0.1), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide2, "Ren_WT_r10k_boundaries_noC_Nora_IAA_pileup_plot", Cm(8.65), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide2, "Ren_WT_r10k_boundaries_noC_WN_BPK25_pileup_plot", Cm(17.3), Cm(0.05), Cm(8), Cm(0.5))
add_text_anno(slide2, "Ren_WT_r10k_boundaries_noC_Ren_WT_pileup_plot", Cm(0.1), Cm(7.24), Cm(8), Cm(0.5))
add_text_anno(slide2, "Ren_WT_r10k_boundaries_noC_Ren_IAA_pileup_plot", Cm(8.65), Cm(7.24), Cm(8), Cm(0.5))

# slide3
slide3 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide3.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Nora_WN_png, Cm(6.23), Cm(0.02), Cm(12.94), Cm(4.31))
slide3.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Ren_WN_png, Cm(6.23), Cm(4.54),  Cm(12.94), Cm(4.31))
slide3.shapes.add_picture(Ren_WT_r10k_boundaries_noC_Nora_WN_Ren_png, Cm(0), Cm(9.04), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide3, "Ren_WT_r10k_boundaries_noC_Nora_WN_pileup_plot", Cm(0.28), Cm(1.92), Cm(7), Cm(0.5))
add_text_anno(slide3, "Ren_WT_r10k_boundaries_noC_Ren_WN_pileup_plot", Cm(0.28), Cm(6.44), Cm(7), Cm(0.5))
add_text_anno(slide3, "Ren_WT_r10k_boundaries_noC_Nora_WN_pileup_plot", Cm(0.28), Cm(8.86), Cm(7), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


In [ ]:
Ren_WT_r10k_boundaries_noC_nora_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_WT_rep2_vs_rep1.png"
Ren_WT_r10k_boundaries_noC_nora_iaa_vs_wt = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Nora_IAA_vs_WT.png"
Ren_WT_r10k_boundaries_noC_wn_bpk25_vs_nora_wt = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25_vs_Nora_WT.png"

Ren_WT_r10k_boundaries_noC_ren_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_WT_rep2_vs_rep1.png"
Ren_WT_r10k_boundaries_noC_ren_iaa_vs_wt = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.Ren_IAA_vs_WT.png"
Ren_WT_r10k_boundaries_noC_wn_bpk25_vs_ren_wt = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.WN_BPK25_vs_Ren_WT.png"

# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')
# slide4
slide4 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide4.shapes.add_picture(Ren_WT_r10k_boundaries_noC_nora_rep2vs1, Cm(0.53), Cm(0), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(Ren_WT_r10k_boundaries_noC_nora_iaa_vs_wt, Cm(9.08), Cm(0), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(Ren_WT_r10k_boundaries_noC_wn_bpk25_vs_nora_wt, Cm(17.73), Cm(0), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(Ren_WT_r10k_boundaries_noC_ren_rep2vs1, Cm(0.53), Cm(7.14), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(Ren_WT_r10k_boundaries_noC_ren_iaa_vs_wt, Cm(9.08), Cm(7.14), Cm(7.14), Cm(7.14))
slide4.shapes.add_picture(Ren_WT_r10k_boundaries_noC_wn_bpk25_vs_ren_wt, Cm(17.73), Cm(7.14), Cm(7.14), Cm(7.14))
# 插入文本注释
add_text_anno(slide4, "Ren_WT_r10k_boundaries_noC_nora_rep2vs1", Cm(0.53), Cm(0), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "Ren_WT_r10k_boundaries_noC_nora_iaa_vs_wt", Cm(9.08), Cm(0), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "Ren_WT_r10k_boundaries_noC_wn_bpk25_vs_nora_wt", Cm(17.73), Cm(0), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "Ren_WT_r10k_boundaries_noC_ren_rep2vs1", Cm(0.53), Cm(7.14), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "Ren_WT_r10k_boundaries_noC_ren_iaa_vs_wt", Cm(9.08), Cm(7.14), Cm(7.14), Cm(0.5))
add_text_anno(slide4, "Ren_WT_r10k_boundaries_noC_wn_bpk25_vs_ren_wt", Cm(17.73), Cm(7.14), Cm(7.14), Cm(0.5))

Ren_WT_r10k_boundaries_noC_nora_wn_3comparation = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_Nora_WN_3comparation.exp_row.png"
Ren_WT_r10k_boundaries_noC_ren_wn_3comparation = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_Ren_WN_3comparation.exp_row.png"
Ren_WT_r10k_boundaries_noC_nora_wn_ren_6comparation = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_noClassified.merge_6comparation.exp_row.png"
# slide5
slide5 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide5.shapes.add_picture(Ren_WT_r10k_boundaries_noC_nora_wn_3comparation, Cm(2.84), Cm(0.57), Cm(19.72), Cm(6.57))
slide5.shapes.add_picture(Ren_WT_r10k_boundaries_noC_ren_wn_3comparation, Cm(2.84), Cm(7.72), Cm(19.72), Cm(6.57))
# 插入文本注释
add_text_anno(slide5, "Ren_WT_r10k_boundaries_noC_nora_wn_3comparation", Cm(0), Cm(0.05), Cm(12.7), Cm(0.5))
add_text_anno(slide5, "Ren_WT_r10k_boundaries_noC_ren_wn_3comparation", Cm(0), Cm(7.14), Cm(12.7), Cm(0.5))

# slide6
slide6 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide6.shapes.add_picture(Ren_WT_r10k_boundaries_noC_nora_wn_ren_6comparation, Cm(2.46), Cm(0.64), Cm(20.48), Cm(13.64))
# 插入文本注释
add_text_anno(slide6, "Ren_WT_r10k_boundaries_noC_nora_wn_ren_6comparation", Cm(0), Cm(0.05), Cm(12.7), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


## add boundaries_CBS2c

In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')
# slide7
slide7 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])

Ren_WT_r10k_boundaries_CBS2c_Nora_WT_rep1_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep1.png"
Ren_WT_r10k_boundaries_CBS2c_Nora_WT_rep2_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep2.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_WT_rep1_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep1.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_WT_rep2_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep2.png"
# slide8
slide8 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide8.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_WT_rep1_png, Cm(0), Cm(0.59), Cm(12.7), Cm(4.23))
slide8.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_WT_rep2_png, Cm(0), Cm(5.38), Cm(12.7), Cm(4.23))
slide8.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_WT_rep1_png, Cm(12.7), Cm(0.59), Cm(12.7), Cm(4.23))
slide8.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_WT_rep2_png, Cm(12.7), Cm(5.38), Cm(12.7), Cm(4.23))
# 插入文本注释
add_text_anno(slide8, "Ren_WT_r10k_boundaries_CBS2c_Nora_WT_rep1_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide8, "Ren_WT_r10k_boundaries_CBS2c_Nora_WT_rep2_png", Cm(0), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide8, "Ren_WT_r10k_boundaries_CBS2c_Ren_WT_rep1_png", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide8, "Ren_WT_r10k_boundaries_CBS2c_Ren_WT_rep2_png", Cm(14.55), Cm(4.85), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_CBS2c_Nora_WT_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT.png"
Ren_WT_r10k_boundaries_CBS2c_Nora_IAA_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_IAA.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_WT_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_IAA_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_IAA.png"
Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25.png"
# slide9
slide9 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide9.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_WT_png, Cm(0), Cm(0.59), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_IAA_png, Cm(0), Cm(5.38), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_png, Cm(0), Cm(10.06), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_WT_png, Cm(12.7), Cm(0.59), Cm(12.7), Cm(4.23))
slide9.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_IAA_png, Cm(12.7), Cm(5.38), Cm(12.7), Cm(4.23))
# 插入文本注释
add_text_anno(slide9, "Ren_WT_r10k_boundaries_CBS2c_Nora_WT_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "Ren_WT_r10k_boundaries_CBS2c_Nora_IAA_png", Cm(0), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_png", Cm(0), Cm(9.53), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "Ren_WT_r10k_boundaries_CBS2c_Ren_WT_png", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide9, "Ren_WT_r10k_boundaries_CBS2c_Ren_IAA_png", Cm(14.55), Cm(4.85), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')

Ren_WT_r10k_boundaries_CBS2c_Nora_WN_row = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN.exp_row.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_WN_row = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WN.exp_row.png"
# slide10
slide10 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide10.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_WN_row, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
slide10.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_WN_row, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
# 插入文本注释
add_text_anno(slide10, "Ren_WT_r10k_boundaries_CBS2c_Nora_WN_row", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide10, "Ren_WT_r10k_boundaries_CBS2c_Ren_WN_row", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_CBS2c_Nora_WN_col = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN.exp_col.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_WN_col = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WN.exp_col.png"
# slide11
slide11 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide11.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_WN_col, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
slide11.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_WN_col, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
# 插入文本注释
add_text_anno(slide11, "Ren_WT_r10k_boundaries_CBS2c_Nora_WN_col", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide11, "Ren_WT_r10k_boundaries_CBS2c_Ren_WN_col", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_CBS2c_Nora_WN_Ren = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WN_Ren.exp_col.png"
# slide12
slide12 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide12.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_WN_Ren, Cm(0.79), Cm(0), Cm(23.81), Cm(14.28))
# 插入文本注释
add_text_anno(slide12, "Ren_WT_r10k_boundaries_CBS2c_Nora_WN_Ren", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')

Ren_WT_r10k_boundaries_CBS2c_Nora_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_WT_rep2_vs_rep1.png"
Ren_WT_r10k_boundaries_CBS2c_Nora_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Nora_IAA_vs_WT.png"
Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_vs_Nora_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25_vs_Nora_WT.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_WT_rep2_vs_rep1.png"
Ren_WT_r10k_boundaries_CBS2c_Ren_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.Ren_IAA_vs_WT.png"
Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_vs_Ren_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.WN_BPK25_vs_Ren_WT.png"
# slide13
slide13 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide13.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_rep2vs1, Cm(0), Cm(0.59), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Nora_IAA_vs_WT, Cm(0), Cm(5.38), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_vs_Nora_WT, Cm(0), Cm(10.06), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_rep2vs1, Cm(12.7), Cm(0.59), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_Ren_IAA_vs_WT, Cm(12.7), Cm(5.38), Cm(12.7), Cm(4.23))
slide13.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_vs_Ren_WT, Cm(12.7), Cm(10.06), Cm(12.7), Cm(4.23))
# 插入文本注释
add_text_anno(slide13, "Ren_WT_r10k_boundaries_CBS2c_Nora_rep2vs1", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "Ren_WT_r10k_boundaries_CBS2c_Nora_IAA_vs_WT", Cm(0), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_vs_Nora_WT", Cm(0), Cm(9.53), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "Ren_WT_r10k_boundaries_CBS2c_Ren_rep2vs1", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "Ren_WT_r10k_boundaries_CBS2c_Ren_IAA_vs_WT", Cm(14.55), Cm(4.85), Cm(10.85), Cm(0.5))
add_text_anno(slide13, "Ren_WT_r10k_boundaries_CBS2c_WN_BPK25_vs_Ren_WT", Cm(14.55), Cm(9.53), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')

Ren_WT_r10k_boundaries_CBS2c_divide_Nora_WN_col = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_Nora_WN_3comparation.exp_col.png"
Ren_WT_r10k_boundaries_CBS2c_divide_Ren_WN_col = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_Ren_WN_3comparation.exp_col.png"
# slide14
slide14 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide14.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_divide_Nora_WN_col, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
slide14.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_divide_Ren_WN_col, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
# 插入文本注释
add_text_anno(slide14, "Ren_WT_r10k_boundaries_CBS2c_divide_Nora_WN_col", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide14, "Ren_WT_r10k_boundaries_CBS2c_divide_Ren_WN_col", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_CBS2c_divide_Nora_WN_Ren = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_CBS2c.merge_6comparation.exp_col.png"
# slide15
slide15 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide15.shapes.add_picture(Ren_WT_r10k_boundaries_CBS2c_divide_Nora_WN_Ren, Cm(0), Cm(0.79), Cm(25.4), Cm(12.7))
# 插入文本注释
add_text_anno(slide15, "Ren_WT_r10k_boundaries_CBS2c_divide_Nora_WN_Ren", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


## add boundaries_sQ4c

In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')
# slide16
slide16 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])

Ren_WT_r10k_boundaries_sQ4c_Nora_WT_rep1_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep1.png"
Ren_WT_r10k_boundaries_sQ4c_Nora_WT_rep2_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep2.png"
# slide17
slide17 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide17.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_WT_rep1_png, Cm(0), Cm(1.31), Cm(25.4), Cm(5.08))
slide17.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_WT_rep2_png, Cm(0), Cm(7.9), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide17, "Ren_WT_r10k_boundaries_sQ4c_Nora_WT_rep1_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide17, "Ren_WT_r10k_boundaries_sQ4c_Nora_WT_rep2_png", Cm(0), Cm(7.15), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_Ren_WT_rep1_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep1.png"
Ren_WT_r10k_boundaries_sQ4c_Ren_WT_rep2_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep2.png"
# slide18
slide18 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide18.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_WT_rep1_png, Cm(0), Cm(1.31), Cm(25.4), Cm(5.08))
slide18.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_WT_rep2_png, Cm(0), Cm(7.9), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide18, "Ren_WT_r10k_boundaries_sQ4c_Ren_WT_rep1_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide18, "Ren_WT_r10k_boundaries_sQ4c_Ren_WT_rep2_png", Cm(0), Cm(7.15), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_Nora_WT_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT.png"
Ren_WT_r10k_boundaries_sQ4c_Nora_IAA_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_IAA.png"
# slide19
slide19 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide19.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_WT_png, Cm(0), Cm(1.31), Cm(25.4), Cm(5.08))
slide19.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_IAA_png, Cm(0), Cm(7.9), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide19, "Ren_WT_r10k_boundaries_sQ4c_Nora_WT_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide19, "Ren_WT_r10k_boundaries_sQ4c_Nora_IAA_png", Cm(0), Cm(7.15), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25.png"
# slide20
slide20 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide20.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_png, Cm(0), Cm(1.31), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide20, "Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_Ren_WT_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT.png"
Ren_WT_r10k_boundaries_sQ4c_Ren_IAA_png = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_IAA.png"
# slide21
slide21 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide21.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_WT_png, Cm(0), Cm(1.31), Cm(25.4), Cm(5.08))
slide21.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_IAA_png, Cm(0), Cm(7.9), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide21, "Ren_WT_r10k_boundaries_sQ4c_Ren_WT_png", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide21, "Ren_WT_r10k_boundaries_sQ4c_Ren_IAA_png", Cm(0), Cm(7.15), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')

Ren_WT_r10k_boundaries_sQ4c_Nora_WN_row = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN.exp_row.png"
# slide22
slide22 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide22.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_WN_row, Cm(0.79), Cm(0), Cm(23.81), Cm(14.29))
add_text_anno(slide22, "Ren_WT_r10k_boundaries_sQ4c_Nora_WN_row", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_Ren_WN_row = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WN.exp_row.png"
# slide23
slide23 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide23.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_WN_row, Cm(0.79), Cm(0), Cm(23.81), Cm(14.29))
add_text_anno(slide23, "Ren_WT_r10k_boundaries_sQ4c_Ren_WN_row", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_Nora_WN_col = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN.exp_col.png"
Ren_WT_r10k_boundaries_sQ4c_Ren_WN_col = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WN.exp_col.png"
# slide24
slide24 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide24.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_WN_col, Cm(2.21), Cm(0), Cm(8.57), Cm(14.29))
slide24.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_WN_col, Cm(14.62), Cm(0), Cm(8.57), Cm(14.29))
# 插入文本注释
add_text_anno(slide24, "Ren_WT_r10k_boundaries_sQ4c_Nora_WN_col", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide24, "Ren_WT_r10k_boundaries_sQ4c_Ren_WN_col", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_Nora_WN_Ren_row = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN_Ren.exp_row.png"
Ren_WT_r10k_boundaries_sQ4c_Nora_WN_Ren_col = "coolpuppy_pileup_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WN_Ren.exp_col.png"
# slide25
slide25 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide25.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_WN_Ren_row, Cm(0), Cm(0.79), Cm(12.7), Cm(12.7))
slide25.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_WN_Ren_col, Cm(12.7), Cm(0.79), Cm(12.7), Cm(12.7))
add_text_anno(slide25, "Ren_WT_r10k_boundaries_sQ4c_Nora_WN_Ren_row", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide25, "Ren_WT_r10k_boundaries_sQ4c_Nora_WN_Ren_col", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')

Ren_WT_r10k_boundaries_sQ4c_Nora_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_WT_rep2_vs_rep1.png"
Ren_WT_r10k_boundaries_sQ4c_Ren_rep2vs1 = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_WT_rep2_vs_rep1.png"
# slide26
slide26 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide26.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_rep2vs1, Cm(0), Cm(1.05), Cm(25.4), Cm(5.08))
slide26.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_rep2vs1, Cm(0), Cm(8.2), Cm(25.4), Cm(5.08))
add_text_anno(slide26, "Ren_WT_r10k_boundaries_sQ4c_Nora_rep2vs1", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide26, "Ren_WT_r10k_boundaries_sQ4c_Ren_rep2vs1", Cm(0), Cm(7.15), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_Nora_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Nora_IAA_vs_WT.png"
Ren_WT_r10k_boundaries_sQ4c_Ren_IAA_vs_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.Ren_IAA_vs_WT.png"
# slide27
slide27 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide27.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Nora_IAA_vs_WT, Cm(0), Cm(1.05), Cm(25.4), Cm(5.08))
slide27.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_Ren_IAA_vs_WT, Cm(0), Cm(8.2), Cm(25.4), Cm(5.08))
add_text_anno(slide27, "Ren_WT_r10k_boundaries_sQ4c_Nora_IAA_vs_WT", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide27, "Ren_WT_r10k_boundaries_sQ4c_Ren_IAA_vs_WT", Cm(0), Cm(7.15), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_vs_Nora_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25_vs_Nora_WT.png"
Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_vs_Ren_WT = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.WN_BPK25_vs_Ren_WT.png"
# slide28
slide28 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide28.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_vs_Nora_WT, Cm(0), Cm(1.05), Cm(25.4), Cm(5.08))
slide28.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_vs_Ren_WT, Cm(0), Cm(8.2), Cm(25.4), Cm(5.08))
add_text_anno(slide28, "Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_vs_Nora_WT", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide28, "Ren_WT_r10k_boundaries_sQ4c_WN_BPK25_vs_Ren_WT", Cm(0), Cm(7.15), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


In [ ]:
# 打开 PPT 并插入图片
my_ppt = Presentation('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')

Ren_WT_r10k_boundaries_sQ4c_divide_Nora_WN_col = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_Nora_WN_3comparation.exp_col.png"
Ren_WT_r10k_boundaries_sQ4c_divide_Ren_WN_col = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_Ren_WN_3comparation.exp_col.png"
# slide14
slide14 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide14.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_divide_Nora_WN_col, Cm(2.26), Cm(0), Cm(8.57), Cm(14.29))
slide14.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_divide_Ren_WN_col, Cm(14.88), Cm(0), Cm(8.57), Cm(14.29))
add_text_anno(slide14, "Ren_WT_r10k_boundaries_sQ4c_divide_Nora_WN_col", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))
add_text_anno(slide14, "Ren_WT_r10k_boundaries_sQ4c_divide_Ren_WN_col", Cm(14.55), Cm(0.05), Cm(10.85), Cm(0.5))

Ren_WT_r10k_boundaries_sQ4c_divide_Nora_WN_Ren = "coolpuppy_pileup_divide_plot_for_Ren_WT_r10k_boundaries_sQ4c.merge_6comparation.exp_col.png"
# slide15
slide15 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
slide15.shapes.add_picture(Ren_WT_r10k_boundaries_sQ4c_divide_Nora_WN_Ren, Cm(4.13), Cm(0), Cm(17.15), Cm(14.29))
add_text_anno(slide15, "Ren_WT_r10k_boundaries_sQ4c_divide_Nora_WN_Ren", Cm(0), Cm(0.05), Cm(10.85), Cm(0.5))

# 保存 PPT
my_ppt.save('8.6_coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries.use_Nora_WN_Ren_mcool.r10k.pptx')


# 移动相关结果到特定文件夹

In [ ]:
%%bash
mkdir coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries_use_e5_r10k
mv coolpuppy_*for_Ren_WT_r10k_boundaries*.pdf ./coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries_use_e5_r10k/
mv coolpuppy_*for_Ren_WT_r10k_boundaries*.png ./coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries_use_e5_r10k/
mv Ren_WT_r10k_insulation_boundaries* ./coolpuppy_pileup_plot_for_Ren_WT_r10k_insulation_boundaries_use_e5_r10k/